In [26]:
import numpy as np

class site:
    def __init__(self, position, properties):
        self.index = position
        for name,value in properties.items():
            setattr(self, name, value)
        

class Ising:
    def __init__(self, inL, inh0):
        self.L = inL
        self.h0 = inh0
        self.system = []
        for i in range(1,self.L):
            self.system.append(site(i, {'Jright':np.random.uniform(0,1), 'h':np.random.uniform(0,self.h0), 'ls':.5, 'lb':.5, 'm':.5}))
        self.system.append(site(self.L-1, {'Jright':0, 'h':np.random.uniform(0,self.h0), 'ls':.5, 'lb':.5, 'm':.5}))
    
    def renormStep(self):
        bond = None
        Omega = 0
        OmegaPos = None
        for i in range(self.L):
            if Omega<self.system[i].Jright:
                Omega = self.system[i].Jright
                OmegaPos = i
                bond = True
            if Omega<self.system[i].h:
                Omega = self.system[i].h
                OmegaPos = i
                bond = False
        
        if bond:
            self.system[OmegaPos].h = self.system[OmegaPos+1].h * self.system[OmegaPos].h / Omega #hright*hleft/Omega
            self.system[OmegaPos].ls = self.system[OmegaPos].ls + self.system[OmegaPos+1].ls + self.system[OmegaPos].lb
            self.system[OmegaPos].lb = self.system[OmegaPos+1].lb
            self.system[OmegaPos].m = self.system[OmegaPos].m + self.system[OmegaPos+1].m
            self.system[OmegaPos].Jright = self.system[OmegaPos+1].Jright
            del self.system[OmegaPos+1]
        elif OmegaPos<(self.L-1) and OmegaPos != 0:
            self.system[OmegaPos-1].Jright = self.system[OmegaPos-1].Jright*self.system[OmegaPos].Jright/Omega #jleft*jright/Omega
            self.system[OmegaPos-1].lb = self.system[OmegaPos-1].lb + self.system[OmegaPos].ls + self.system[OmegaPos].lb
            del self.system[OmegaPos]
        elif OmegaPos == 0:
            del self.system[OmegaPos]
        else:
            self.system[OmegaPos-1].Jright = 0
            del self.system[OmegaPos]
        self.L -= 1
        
    #Analytics
    def printCouplings(self):
        for i in range(self.L-1):
              print('h' + str(i) + ' = ' + str(self.system[i].h) + '\nJ' + str(i) + ',' + str(i+1) + ' = ' + str(self.system[i].Jright))
        print('h' + str(self.L-1) + ' = ' + str(self.system[self.L-1].h))
        print()
    
    def printDistributions(self):
        hdist = []
        jdist = []
        lsdist = []
        lbdist = []
        for i in range(self.L-1):
            hdist.append(self.system[i].h)
            jdist.append(self.system[i].Jright)
            lsdist.append(self.system[i].ls)
            lbdist.append(self.system[i].lb)
        hdist.append(self.system[self.L-1].h)
        lsdist.append(self.system[self.L-1].ls)
        hdistlog = np.log(hdist)
        jdistlog = np.log(jdist)
        print('log average h is: ' + str(np.mean(hdistlog)))
        print('log h variance is: ' + str(np.var(hdistlog)) + '\n')
        print('log average J is: ' + str(np.mean(jdistlog)))
        print('log J variance is: ' + str(np.var(jdistlog)) + '\n')
        print('average spin cluster size is: ' + str(np.mean(lsdist)) + '\n')
        print('average bond length is: ' + str(np.mean(lbdist)) + '\n')
    
        
        
        
        
L0 = 2000
tester = Ising(L0,1)

#tester.printCouplings()
for i in range(L0-50):
    tester.renormStep()
tester.printCouplings()
tester.printDistributions()

h0 = 0.00016972999506738597
J0,1 = 0.001235723020133846
h1 = 0.00017482148145498932
J1,2 = 3.2312321968321984e-06
h2 = 7.641879260596789e-07
J2,3 = 1.5803391088093084e-05
h3 = 4.045786345923844e-07
J3,4 = 4.933890942687209e-10
h4 = 0.0007539311578929515
J4,5 = 1.5656638178666664e-05
h5 = 0.0010070959645500268
J5,6 = 6.832551875566251e-07
h6 = 6.616450537940512e-06
J6,7 = 0.004808872708311845
h7 = 8.481413723154513e-06
J7,8 = 0.0013056461728522724
h8 = 0.0005597911958275846
J8,9 = 0.004333155592621019
h9 = 0.0010535222259705514
J9,10 = 0.0010995179043979443
h10 = 1.5578324402623144e-05
J10,11 = 4.505759189241803e-06
h11 = 1.5169566774721268e-07
J11,12 = 0.00044450672318240345
h12 = 3.83526262133646e-05
J12,13 = 0.0018622893426005073
h13 = 1.1545171738646718e-06
J13,14 = 0.004795422753888404
h14 = 0.0008086648473461462
J14,15 = 7.740071966847345e-05
h15 = 1.2873625222614597e-06
J15,16 = 8.353631642424281e-05
h16 = 0.0018871742747149243
J16,17 = 7.164110180142427e-09
h17 = 1.0309215683515